In [1]:
import pandas as pd
import ipywidgets as widgets
import datetime as dt
from datetime import datetime
from IPython.display import display, clear_output
from IPython.core.display import HTML
import random
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import tempfile
import numpy as np
from functools import partial
import re

In [2]:
#TODO: Formatar o main app, modificando o seletor de funcoes

In [3]:
pd.set_option("display.max_rows", None)

In [4]:
# AVISO PROMOTOR E PRACA ATIVA NAO FUNCIONA 

In [5]:
class app:
    def __init__(self, main) -> None:
        self.main = main
        
    def _build_layout(self):
        self.output_main = widgets.Output()
        self.layout = widgets.VBox(children=[self.output_main])
        return self.layout
    
    def _filtrar_pedidos(self, status = None, statusIs = '=='):
        self._atualizar_filtros()
        pedidos = self.main.pedidos.query(
            f"@self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_praca in @self.pracas")
        if status != None:
            pedidos = pedidos.query(f"status {statusIs} @status")
        return pedidos
    
    def _filtrar_artigosNoPedido(self, status = "FECHADO"):
        self._atualizar_filtros()
        artigosNoPedido =     self.main.artigosPedido.query(
            f"ref in @self.artigosFiltrar and @self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_praca in @self.pracas and status == @status"
        )
        return artigosNoPedido
        
    def _filtrar_desempenho(self):
        self._atualizar_filtros()
        desempenho = self.main.desempenhoPraca.query("@self.dataInicial <= ano_mes_cobranca <= @self.dataFinal  and id_vendedor in @self.promotores and id_praca in @self.pracas")
        return desempenho
    
    def _get_listaVendedorasAtivas(self):
        pedidos = self._filtrar_pedidos('DIGITADO')
        lista_vendedorasAtivas = pedidos['id_sacoleira'].tolist()
        return lista_vendedorasAtivas
    
    def _get_itensVendedorasAtivas(self):
        listaAtiva = self._get_listaVendedorasAtivas()
        histArtigosAtivas = self.main.artigosPedido.query("id_vendedora in @listaAtiva")
        return histArtigosAtivas
    
    def _atualizar_exibicao(self, exibicoes, output=None):
        """
        Args:
            exibicoes (list): Uma lista contendo um ou mais dicionario de exibicao,
            exibicao (dic): Objeto a ser exibido em uma tab sendo seus argumentos:
            * 'nome' : Nome da tab;
            * 'objeto' : Objeto a ser exibido;
            * 'style' : parametros de estilo para dataframes;
            output (_type_, optional): _description_. Defaults to None.
        """
        if output is not None:
            output = output
        else:
            output = self.output_main

        if not isinstance(exibicoes, list):
            exibicoes = [exibicoes]
            
        with output:
            clear_output()
            tab_contents = [widgets.Output() for _ in range(len(exibicoes))]
            children = [widgets.VBox([tab]) for tab in tab_contents]
            tab = widgets.Tab()
            tab.children = children
            for i, extra in enumerate(exibicoes):
                tab.set_title(i, exibicoes[i]["nome"])
            display(tab)

            for i, exibicao in enumerate(exibicoes):
                with tab_contents[i]:
                    if isinstance(exibicao['objeto'], list):
                        for item in exibicao['objeto']:
                            display(item)
                    elif isinstance(exibicao['objeto'], folium.Map):
                        display(HTML(f'<div style="width: 1600px; height: 700px;">{exibicao["objeto"]._repr_html_()}</div>'))
                    elif isinstance(exibicao['objeto'], pd.DataFrame):
                        def criar_funcao_atualizar(exibicao, dropdown_coluna, dropdown_ascDesc, output):
                            def atualizar_dataframe(change):
                                dataframe = exibicao['objeto']
                                colunas = [coluna for coluna in exibicao['objeto'].columns]
                                if dropdown_coluna.value in colunas:
                                    dataframe_final = dataframe.sort_values(by=dropdown_coluna.value, ascending=dropdown_ascDesc.value)
                                else:
                                    dataframe_final = dataframe.sort_index(level=dropdown_coluna.value, ascending=dropdown_ascDesc.value)
                                if "style" in exibicao:
                                    dataframe_final = dataframe_final.style.format(exibicao['style'])
                                with output:
                                    clear_output()
                                    display(dataframe_final)
                            return atualizar_dataframe
                        def criar_exportar_exibicao(objeto):
                            def exportar_exibicao(objeto):
                                agora = datetime.now()
                                export = agora.strftime('%Y-%m-%d_%H-%M-%S')
                                dataframe = exibicao['objeto']
                                dataframe.to_excel(f"export/dataframe_exportado_{export}.xlsx", index=True)
                                
                        exibicao = exibicao
                        colunas_exb = []
                        for index in exibicao['objeto'].index.names:
                            colunas_exb.append(index)
                        for coluna in exibicao['objeto'].columns:
                            colunas_exb.append(coluna)
                        dropdown_coluna_exibicao = widgets.Dropdown(description='Campo', options=colunas_exb)
                        dropdown_ascDesc_exibicao = widgets.Dropdown(description='Orientação', options=[('Ascendente', True), ('Descendente', False)])
                        button_ordenar_exibicao = widgets.Button(description='Ordenar')
                        box_ordem_exibicao = widgets.HBox(children=[dropdown_coluna_exibicao, dropdown_ascDesc_exibicao, button_ordenar_exibicao])
                        button_exportar_exibicao = widgets.Button(description="Exportar")
                        output_exibicao = widgets.Output()
                        box_main = widgets.VBox(children=[box_ordem_exibicao, button_exportar_exibicao, output_exibicao])
                        funcao_atualizar_dataframe = criar_funcao_atualizar(exibicao, dropdown_coluna_exibicao, dropdown_ascDesc_exibicao, output_exibicao)
                        funcao_exportar = criar_exportar_exibicao(exibicao['objeto'])
                        button_ordenar_exibicao.on_click(funcao_atualizar_dataframe)
                        button_exportar_exibicao.on_click(funcao_exportar)
                        display(box_main)
                        funcao_atualizar_dataframe(None)  # O segundo argumento (change) será preenchido automaticamente pelo widget
                    else:
                        display(exibicao['objeto'])
    
    def _atualizar_filtros(self):
        self.artigosFiltrar = self.main.filtros['artigos']
        self.dataInicial = self.main.filtros['data inicial']
        self.dataFinal = self.main.filtros['data final']
        self.promotores = self.main.filtros['promotores']
        self.pracas = self.main.filtros['pracas']
        
    def _retornar_funcoes(self):
        def normalize(funcao_chamavel):
            normalize = funcao_chamavel.replace("_", " ")
            normalize = normalize.title()
            return normalize
        atributos = dir(self)
        todas_funcoes = [atributo for atributo in atributos if callable(getattr(self, atributo))]
        funcoes_externas = [funcao for funcao in todas_funcoes if funcao[0] != '_']
        funcoes_chamaveis = []
        for funcao in funcoes_externas:
            funcoes_chamaveis.append((normalize(funcao), funcao))
        return funcoes_chamaveis
        

In [6]:
class relatorio(app):
    def __init__(self, main) -> None:
        super().__init__(main)
    
    def venda_periodo(self):
        pedidos = self._filtrar_pedidos().groupby('status')
        df = pd.DataFrame()
        df['Total Consignado'] = pedidos['valor_total_pedido'].sum()
        df['Total Pedidos'] = pedidos['id_pedido'].count()
        df['Total Vendido'] = pedidos['valor_venda_pedido'].sum()
        
        style = {
            'Total Consignado': 'R$ {:,.2f}',
            'Total Pedidos': '{:.0f}',
            'Total Vendido': 'R$ {:,.2f}'
        }
        self._atualizar_exibicao({'nome':'Relatorio', 'objeto':df, 'style': style})
    
    def total_na_rua(self):
        pedidos = self._filtrar_pedidos().groupby('status')
        df = pd.DataFrame()
        df['Total Consignado'] = pedidos['valor_total_pedido'].sum()
        df['Total Pedidos'] = pedidos['id_pedido'].count()
        
        style = {
            'Total Consignado': 'R$ {:,.2f}',
            'Total Pedidos': '{:.0f}'
        }
        self._atualizar_exibicao({'nome':'Relatorio', 'objeto':df, 'style': style})
        
        
    def top_10_artigos_venda(self):
        artigos = self._filtrar_artigosNoPedido().groupby(['ref', 'artigo']).agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', 'valor_vendido': 'sum'})
        top10 = artigos.nlargest(10, 'quantidade_vendida')
        style = {
            'valor_vendido': 'R$ {:,.2f}',
            'quantidade_entregue': '{:.0f}',
            'quantidade_vendida': '{:.0f}',
            'Rep Faturamento' : '{:.2%}'
        }
        self._atualizar_exibicao({'nome':'Relatorio', 'objeto':top10})

    def top_10_artigos_faturamento(self):
        artigos = self._filtrar_artigosNoPedido().groupby(['ref', 'artigo']).agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', 'valor_vendido': 'sum'})
        top10 = artigos.nlargest(10, 'valor_vendido')
        top10['Rep Faturamento'] = top10['valor_vendido'] / artigos['valor_vendido'].sum()
        style = {
            'valor_vendido': 'R$ {:,.2f}',
            'quantidade_entregue': '{:.0f}',
            'quantidade_vendida': '{:.0f}',
            'Rep Faturamento' : '{:.2%}'
        }
        self._atualizar_exibicao({'nome':'Relatorio', 'objeto':top10, 'style': style})
        
    def dados_itens_fora_top_10(self):
        artigos = self._filtrar_artigosNoPedido().groupby(['ref', 'artigo']).agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', 'valor_vendido': 'sum'})
        artigos.reset_index(inplace=True)  # Certifique-se de atribuir de volta a "artigos" após reset_index
        top10 = artigos.nlargest(10, 'valor_vendido')
        listaTop10 = top10['ref'].tolist()
        
        # Filtrar artigos que não estão no top 10
        artigos_fora_top10 = artigos[~artigos['ref'].isin(listaTop10)]
        artigos_fora_top10 = pd.merge(artigos_fora_top10, self.main.artigos[['ref', 'descricao_grupo']], on='ref')
        artigos_fora_top10g = artigos_fora_top10.groupby('descricao_grupo')
        
        # Grupos fora 
        df = pd.DataFrame()
        df['Quantidade de Itens'] = artigos_fora_top10g['ref'].count()
        df['Total Vendido R$'] = artigos_fora_top10g['valor_vendido'].sum()
        df['Total Un Vendidos'] = artigos_fora_top10g['quantidade_vendida'].sum()
        df['Rep Total Vendido'] = df['Total Vendido R$'] / artigos['valor_vendido'].sum()
        df['Total Consignado'] = artigos_fora_top10g['quantidade_entregue'].sum()
        df['% Vendida'] = df['Total Un Vendidos']/df['Total Consignado']
        
        style = {
            'Total Vendido R$': 'R$ {:,.2f}',
            'Total Un Vendidos': '{:.0f}',
            'Quantidade de Itens': '{:.0f}',
        }
        
        self._atualizar_exibicao({'nome': 'Relatorio', 'objeto': df, 'style': style})
    
    def analise_grupos(self):
        artigos = self._filtrar_artigosNoPedido().groupby(['ref', 'artigo']).agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', 'valor_vendido': 'sum'})
        artigos.reset_index(inplace=True)  # Certifique-se de atribuir de volta a "artigos" após reset_index
        artigos = pd.merge(artigos, self.main.artigos[['ref', 'descricao_grupo']], on='ref')
        grupos = artigos.groupby('descricao_grupo')
        
        df = pd.DataFrame()
        df['Quantidade de Itens'] = grupos['ref'].count()
        df['Total Vendido R$'] = grupos['valor_vendido'].sum()
        df['Total Un Vendidos'] = grupos['quantidade_vendida'].sum()
        df['Rep Total Vendido'] = df['Total Vendido R$'] / artigos['valor_vendido'].sum()
        df['Total Consignado'] = artigos['quantidade_entregue'].sum()
        df['% Vendida'] = df['Total Un Vendidos']/df['Total Consignado']
        
        self._atualizar_exibicao({'nome': 'Relatorio', 'objeto': df})
        
    def desempenho_periodo_promotor(self):
        desempenhos = self._filtrar_desempenho()
        gp_desempenhos = desempenhos.groupby('descricao_vendedor')
        df = pd.DataFrame()
        df['Praças Cobradas'] = gp_desempenhos['id_desempenhopraca'].count()
        df['Total Recebido'] = gp_desempenhos['valor_recebido'].sum()
        df['Total Pepinos'] = gp_desempenhos['total_novos_pepinos'].sum()
        df['Vendedoras Cobradas'] = gp_desempenhos['pedidos_fechados'].sum()
        df['Recebimento x Acertos'] = df['Total Recebido'] / df['Praças Cobradas']
        df['Total Consignado'] = gp_desempenhos['vl_total_pedidos_liquido'].sum()
        self._atualizar_exibicao({'nome': 'Relatorio', 'objeto': df})
        
    def exprtavel_cidades_por_praca(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        pedidos = self._filtrar_pedidos()
        with pd.ExcelWriter(f"export/cidades-pracas_{export}.xlsx", engine='xlsxwriter') as writer:
            for promotor in pedidos['descricao_vendedor'].unique():
                pedidosPromotor = pedidos[pedidos['descricao_vendedor']==promotor]
                pedidosPromotorGP = pedidosPromotor.groupby(['descricao_praca', 'cidade'])
                df = pd.DataFrame()
                df['Pedidos'] = pedidosPromotorGP['id_pedido'].count()
                df['Total Deixado'] = pedidosPromotorGP['valor_total_pedido'].sum()
                df['Total Vendido'] = pedidosPromotorGP['valor_venda_pedido'].sum()
                df.to_excel(writer, sheet_name=f'{promotor[:-8]}', index=True)
        
    def exprtavel_cidades(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        pedidos = self._filtrar_pedidos()
        with pd.ExcelWriter(f"export/cidades_{export}.xlsx", engine='xlsxwriter') as writer:
            for promotor in pedidos['descricao_vendedor'].unique():
                pedidosPromotor = pedidos[pedidos['descricao_vendedor']==promotor]
                pedidosPromotorGP = pedidosPromotor.groupby(['cidade'])
                df = pd.DataFrame()
                df['Pedidos'] = pedidosPromotorGP['id_pedido'].count()
                df['Total Deixado'] = pedidosPromotorGP['valor_total_pedido'].sum()
                df['Total Vendido'] = pedidosPromotorGP['valor_venda_pedido'].sum()
                df.to_excel(writer, sheet_name=f'{promotor[:-8]}', index=True)
                
    def exportavel_so_cidades(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        pedidos = self._filtrar_pedidos()
        with pd.ExcelWriter(f"export/SO_cidades_{export}.xlsx", engine='xlsxwriter') as writer:
            gp = pedidos.groupby('cidade')
            df = pd.DataFrame()
            df['Pedidos'] = gp['id_pedido'].count()
            df['Total Deixado'] = gp['valor_total_pedido'].sum()
            df['Total Vendido'] = gp['valor_venda_pedido'].sum()
            df.to_excel(writer, sheet_name=f'cidades', index=True)
            
    def exportavel_desempenho_promotor_por_mes(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        desempenho = self._filtrar_desempenho()
        with pd.ExcelWriter(f"export/desempenho_promotor_{export}.xlsx", engine='xlsxwriter') as writer:
            for promotor in desempenho['descricao_vendedor'].unique():
                df = pd.DataFrame()
                desempenhoPromotor = desempenho[desempenho['descricao_vendedor'] == promotor]
                gp = desempenhoPromotor.groupby('ano_mes_cobranca')
                df['Pedidos Cobrados'] = gp['pedidos_fechados'].sum()
                df['Clientes novos'] = gp['clientes_novos'].sum()
                df['Deixado'] = gp['vl_total_pedidos_liquido'].sum()
                df['Cobrança'] = gp['valor_recebido'].sum()
                df['Pepinos'] = gp['total_novos_pepinos'].sum()
                df['Rec Pepino'] = gp['valor_pepino_recebidos'].sum()
                df.to_excel(writer, sheet_name=f'{promotor[:-8]}', index=True)
                
    def exportavel_desempenho_promotor_geral(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        desempenho = self._filtrar_desempenho()
        with pd.ExcelWriter(f"export/desempenho_promotores_{export}.xlsx", engine='xlsxwriter') as writer:
            df = pd.DataFrame()
            desempenhoPromotor = desempenho
            gp = desempenhoPromotor.groupby('descricao_vendedor')
            df['Pedidos Cobrados'] = gp['pedidos_fechados'].sum()
            df['Clientes novos'] = gp['clientes_novos'].sum()
            df['Deixado'] = gp['vl_total_pedidos_liquido'].sum()
            df['Cobrança'] = gp['valor_recebido'].sum()
            df['Pepinos'] = gp['total_novos_pepinos'].sum()
            df['Rec Pepino'] = gp['valor_pepino_recebidos'].sum()
            df.to_excel(writer, sheet_name=f'tabela', index=True)
                

In [7]:
class MainApp:
    def __init__(self):
        self.relatorios = relatorio(self)
        self.apps = [('Relatorios', self.relatorios)]
        self.load_data()
        self.create_layout()         
        self.tabs_data = []
        
    def load_data(self):
        self.vendedores = pd.read_csv('data/vendedores.csv', sep=';')
        self.pracas = pd.read_csv('data/pracas.csv', sep=';')
        self.desempenhoPraca = pd.read_csv('data/desempenhoPraca.csv', sep=';')
        self.pedidos = pd.read_csv('data/pedidos.csv', sep=';')
        self.sacoleiras = pd.read_csv('data/sacoleiras.csv', sep=';')
        self.artigosPedido = pd.read_csv('data/artigosPedido.csv', sep=';')
        self.gruposArtigos = pd.read_csv('data/grupos.csv', sep=';')
        self.artigos = pd.read_csv('data/artigos.csv', sep=';')
        self.infoCidades = pd.read_csv('data/cidades.csv', sep=';')
        
        self.artigosPedido['ano_mes_cobranca'] = pd.to_datetime(self.artigosPedido['ano_mes_cobranca'], format='%m/%Y').dt.date
        self.desempenhoPraca['ano_mes_cobranca'] = pd.to_datetime(self.desempenhoPraca['ano_mes_cobranca'], format='%Y-%m').dt.date
        self.pedidos['ano_mes_cobranca'] = pd.to_datetime(self.pedidos['ano_mes_cobranca'], format='%m/%Y').dt.date

    def create_layout(self):
        # Promotor
        self.check_soPromotorEPracaAtivo = widgets.Checkbox(description="Só Ativas?", value=True)
        listaPromotores = self.get_lista_promotores()
        self.select_promotores = widgets.SelectMultiple(description='Promotor(es)', options=listaPromotores, value=[promotor[1] for promotor in listaPromotores])
        self.check_todosPromotores = widgets.Checkbox(description="Todos", value=False)
        self.box_promotores = widgets.HBox(children=[self.select_promotores, self.check_todosPromotores])
        
        # Praça
        listaPracas = self.get_lista_pracas()
        self.text_id_pracas = widgets.Text(description='Praça(s) [id]')
        self.select_pracas = widgets.SelectMultiple(description='Praça(s)', options=listaPracas, value=[praca[1] for praca in listaPracas])
        self.check_todasPracas = widgets.Checkbox(description="Todas", value=False)
        self.box_pracas = widgets.HBox(children=[self.text_id_pracas, self.select_pracas, self.check_todasPracas])
        self.box_promotorPraca = widgets.HBox(children=[self.box_promotores, self.box_pracas, self.check_soPromotorEPracaAtivo])
        
        # Periodo
        self.dateP_dataInicial = widgets.DatePicker(description='Data Inicial Cobrança', value=dt.date.today() - dt.timedelta(days=2 * 30))
        self.dateP_dataFinal = widgets.DatePicker(description='Data Final Cobrança', value=dt.date.today() + dt.timedelta(days=2 * 30))
        self.box_periodo = widgets.HBox(children=[self.dateP_dataInicial, self.dateP_dataFinal])
        
        # Grupos e Artigos
        listaGrupos = self.get_lista_grupos()
        self.select_gruposArtigos = widgets.SelectMultiple(description='Grupos', options=listaGrupos, value=[grupo[1] for grupo in listaGrupos])
        self.check_todosGrupos = widgets.Checkbox(description='Todos', value=False)
        listaArtigos = self.get_lista_artigos()
        self.text_ref_artigos = widgets.Text(description='Artigos(s) [ref]')
        self.select_artigos = widgets.SelectMultiple(description='Artigos', options=listaArtigos, value=[artigo[1] for artigo in listaArtigos])
        self.check_todosArtigos = widgets.Checkbox(description='Todos', value=False)
        self.box_gruposArtigos = widgets.HBox(children=[self.select_gruposArtigos, self.check_todosGrupos,self.text_ref_artigos, self.select_artigos, self.check_todosArtigos])

        # Relatórios
        self.select_app = widgets.Dropdown(description='App:', options=self.apps)
        self.select_funcao = widgets.Dropdown(description='Função:', options=self.get_lista_funcoes())
        self.togle_modoExibicao = widgets.ToggleButtons(
            options=['Mesma Tab', 'Nova Tab'],
            description='Relatório na:',
            value='Mesma Tab',
            tooltips=['O relatório substituirá a tab atual', 'O relatório entrará em uma nova tab'],
        )
        self.button_rodar = widgets.Button(description= 'Rodar')
        self.button_rodar.on_click(self._rodar)
        self.button_fechar = widgets.Button(description= 'Fechar Tab')
        self.button_fechar.on_click(self.fecharTabAtiva)
        self.box_apps = widgets.HBox(children=[self.select_app, self.select_funcao, self.togle_modoExibicao, self.button_rodar, self.button_fechar])

        self.filtros = [self.box_promotorPraca, self.box_periodo, self.box_gruposArtigos]
        self.box_filtros = widgets.VBox(children=self.filtros)
        self.tab_apps = widgets.Tab()
        self.box_fixos = [self.box_filtros, self.box_apps, self.tab_apps]
        self.box_main = widgets.VBox(children=self.box_fixos)
        # Add observers to all widgets
        self.add_observer_to_widgets(self.box_filtros)
        # Display the nested boxes
        display(self.box_main)
    
    def get_lista_funcoes(self):
        chamado = getattr(self.select_app.value, '_retornar_funcoes')
        lista = chamado()
        return lista
    
    def get_lista_promotores(self):
        listaPromotores = []
        promotores = self.vendedores.sort_values(by='descricao_vendedor', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            promotores = promotores.query('status_vendedor == "ATIVO"')
        for _, linha in promotores.iterrows():
            tupla = (linha['descricao_vendedor'], linha['id_vendedor'])
            listaPromotores.append(tupla)
        return listaPromotores
    
    #TODO Juntar função get_lista_pracas e get_lista_id_pracas
    def get_lista_pracas(self):
        listaPracas = []
        promotores = self.select_promotores.value
        pracas = self.pracas.query('id_vendedor.isin(@promotores)')
        pracas = pracas.sort_values(by='descricao_praca', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            pracas = pracas.query('status_praca == "ATIVO"')
        for _, linha in pracas.iterrows():
            tupla = (linha['descricao_praca'], linha['id_praca'])
            listaPracas.append(tupla)
        return listaPracas
    
    def _so_promotor_praca_ativo(self, _):
        self.select_promotores.options = self.get_lista_promotores()
    
    def get_lista_id_pracas(self):
        listaPracas = []
        promotores = self.select_promotores.value
        pracas = self.pracas.query('id_vendedor.isin(@promotores)')
        pracas = pracas.sort_values(by='descricao_praca', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            pracas = pracas.query('status_praca == "ATIVO"')
        for _, linha in pracas.iterrows():
            tupla = (linha['id_praca'])
            listaPracas.append(tupla)
        return listaPracas

    def get_lista_grupos(self):
        listaGrupos = []
        grupos = self.gruposArtigos
        grupos = grupos.sort_values(by='descricao_grupo', ascending=True)

        for _, linha in grupos.iterrows():
            tupla = (linha['descricao_grupo'], linha['id_grupo'])
            listaGrupos.append(tupla)

        return listaGrupos
    
    #TODO Juntar função get_lista_artigos e get_lista_ref_artigos
    def get_lista_artigos(self):
        listaArtigos = []
        grupos = self.select_gruposArtigos.value
        artigos = self.artigos.query('id_grupo.isin(@grupos)')
        artigos = artigos.sort_values(by='artigo', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            artigos = artigos.query('status_artigo == "ATIVO"')
        for _, linha in artigos.iterrows():
            tupla = (linha['artigo'], linha['ref'])
            listaArtigos.append(tupla)
        return listaArtigos
    
    def get_lista_ref_artigos(self):
        listaArtigos = []
        grupos = self.select_gruposArtigos.value
        artigos = self.artigos.query('id_grupo.isin(@grupos)')
        artigos = artigos.sort_values(by='artigo', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            artigos = artigos.query('status_artigo == "ATIVO"')
        for _, linha in artigos.iterrows():
            tupla = (linha['ref'])
            listaArtigos.append(tupla)
        return listaArtigos

    def add_observer_to_widgets(self, widget):
        # Specific functions
        self.check_soPromotorEPracaAtivo.observe(self._so_promotor_praca_ativo, names='value') #type: ignore
        self.select_promotores.observe(self._promotor_selecionado, names='value') # type: ignore
        self.select_gruposArtigos.observe(self._grupos_selecionados, names='value') # type: ignore
        self.check_todosPromotores.observe(self._selecionar_todos_promotores, names='value') # type: ignore
        self.check_todasPracas.observe(self._selecionar_todas_pracas, names='value') # type: ignore
        self.check_todosGrupos.observe(self._selecionar_todos_grupos, names='value') # type: ignore
        self.check_todosArtigos.observe(self._selecionar_todos_artigos, names='value') # type: ignore
        self.select_app.observe(self._app_selecionado, names='value') # type: ignore
        self.text_id_pracas.observe(self.id_praca_escrito, names='value') # type: ignore
        self.text_ref_artigos.observe(self.ref_artigo_escrito, names='value') # type: ignore
        
        # General function
        if isinstance(widget, (widgets.Box, widgets.VBox, widgets.HBox, widgets.GridBox)):
            for child_widget in widget.children: # type: ignore
                self.add_observer_to_widgets(child_widget)
        elif isinstance(widget, widgets.Widget):
            widget.observe(self.normal_object_updated, names='value') # type: ignore

    def normal_object_updated(self, change):
        if change['new'] == ():
            pass
        else:
            self._update_termos()

    def id_praca_escrito(self, change):
        valor = change['new']
        if valor == '':
            return
        padrao = re.compile(r'^[0-9,]*$')
        lista_pracas = self.get_lista_id_pracas()
        if padrao.match(valor):
            if valor[-1] != ',':
                pracasSelecionadasLista = [int(numero) for numero in valor.split(',')]
                pracasSelecionadas = []
                for praca in pracasSelecionadasLista:
                    if praca in lista_pracas:
                        pracasSelecionadas.append(praca)
                self.select_pracas.value = pracasSelecionadas
        else:
            self.text_id_pracas.value = change['old']
        
    def ref_artigo_escrito(self, change):
        valor = change['new']
        if valor == '':
            return
        padrao = re.compile(r'^[0-9,]*$')
        lista_artigos = self.get_lista_ref_artigos()
        if padrao.match(valor):
            if valor[-1] != ',':
                artigosSelecionadosLista = [int(numero) for numero in valor.split(',')]
                artigosSelecionados = []
                for praca in artigosSelecionadosLista:
                    if praca in lista_artigos:
                        artigosSelecionados.append(praca)
                self.select_artigos.value = artigosSelecionados
        else:
            self.text_ref_artigos.value = change['old']
        
    def _update_termos(self):
        # Convertendo as strings das datas para o formato "mm/aaaa"
        data_inicial_str = self.dateP_dataInicial.value.strftime("%m/%Y") # type: ignore
        data_final_str = self.dateP_dataFinal.value.strftime("%m/%Y") # type: ignore
        # Convertendo as strings para o tipo datetime
        data_inicial = datetime.strptime(data_inicial_str, "%m/%Y").date()
        data_final = datetime.strptime(data_final_str, "%m/%Y").date()
        self.filtros = {
            'promotores': self.select_promotores.value,
            'pracas': self.select_pracas.value,
            'data inicial': data_inicial,
            'data final': data_final,
            'artigos': self.select_artigos.value
        }
    
    def _updateOptionsPracas(self):
        self.select_pracas.options = self.get_lista_pracas()
        self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]

    def _updateOptionsArtigos(self):
        self.select_artigos.options = self.get_lista_artigos()
        self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]

    def _updateOptionsFuncoes(self):
        self.select_funcao.options = self.get_lista_funcoes()

    def _promotor_selecionado(self, change):
        self._updateOptionsPracas()

    def _grupos_selecionados(self, change):
        self._updateOptionsArtigos()

    def _app_selecionado(self, change):
        self._updateOptionsFuncoes()
        self._update_termos()

    def _rodar(self, _):
        def rodarAppNaTabAtiva(self):
            indice_tab_ativa = self.tab_apps.selected_index
            
            # Verifica se há alguma tab aberta
            if indice_tab_ativa is not None and indice_tab_ativa >= 0:
                tab_ativa = self.tab_apps.children[indice_tab_ativa] # type: ignore
                selected_app = self.select_app.value
                app_widgets = selected_app._build_layout() # type: ignore
                tab_ativa.children = [] + [app_widgets]
                funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
                funcao_selecionada()
                self.tab_apps.set_title(len(self.tab_apps.children) - 1, f'{self.select_funcao.label}') # type: ignore
            else:
                rodarAppEmNovaTab(self)
                
        def rodarAppEmNovaTab(self):
            selected_app = self.select_app.value
            app_widgets = selected_app._build_layout() # type: ignore
            nova_aba = widgets.VBox([app_widgets])
            self.tab_apps.children = list(self.tab_apps.children) + [nova_aba] # type: ignore
            self.tab_apps.set_title(len(self.tab_apps.children) - 1, f'{self.select_funcao.label}')
            funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
            funcao_selecionada()
                
        self._update_termos()
        if self.togle_modoExibicao.value == 'Mesma Tab':
            rodarAppNaTabAtiva(self)
        elif self.togle_modoExibicao.value == 'Nova Tab':
            rodarAppEmNovaTab(self)
    
    def fecharTabAtiva(self, _):
        indice_tab_ativa = self.tab_apps.selected_index
        if indice_tab_ativa is not None and indice_tab_ativa >= 0:
            self.tab_apps.children = [
            child for i, child in enumerate(self.tab_apps.children) if i != indice_tab_ativa # type: ignore
        ]
        
    def _selecionar_todos_promotores(self, change):
        if change['new']:
            self.select_promotores.value = [promotor[1] for promotor in self.get_lista_promotores()]
        else:
            self.select_promotores.value = []

    def _selecionar_todas_pracas(self, change):
        if change['new']:
            self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]
        else:
            self.select_pracas.value = []

    def _selecionar_todos_grupos(self, change):
        if change['new']:
            self.select_gruposArtigos.value = [grupo[1] for grupo in self.get_lista_grupos()]
        else:
            self.select_gruposArtigos.value = []

    def _selecionar_todos_artigos(self, change):
        if change['new']:
            self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]
        else:
            self.select_artigos.value = []


21 - 01/09/2020 - 01/08/2021 ~
22 - 01/09/2021 - 01/08/2022 ~
23 - 01/09/2022 - 01/08/2023

In [8]:
oi = MainApp()

In [9]:
pd.set_option('display.max_columns', None)


In [10]:
oi.pedidos.head()

,id_pedido,ano_sem_pedido,ano_sem_cobranca,id_praca,id_sacoleira,status,quantidade_entregue,valor_total_sem_comissao,valor_total_com_comissao,valor_total_pedido,data_retorno,id_vendedor,descricao_praca,descricao_vendedor,cidade,uf,latitude,longitude,ano_sem_pedido_data,ano_mes_pedido,ano_pedido,ano_sem_cobranca_data,ano_mes_cobranca,ano_cobranca,quant_vendida,valor_venda_pedido,valor_liquido
0,1860,20/10,20/18,5,1743,FECHADO,87,0,1775.1,1775.1,NaN,1,ITAPETININGA - 1 - CLJ .,CLEBER JOSE MEDINA CUNHA (CLJ),CAPELA DO ALTO,SP,-23.4685,-47.7388,2020-03-09,03/2020,2020,2020-05-04,2020-05-01,2020,10.0,156.7,109.69
1,1882,20/18,20/25,5,1743,FECHADO,11,0,764.1,764.1,2020-06-08,1,ITAPETININGA - 1 - CLJ .,CLEBER JOSE MEDINA CUNHA (CLJ),CAPELA DO ALTO,SP,-23.4685,-47.7388,2020-05-04,05/2020,2020,2020-06-22,2020-06-01,2020,0.0,0.0,0.00
2,8061,20/25,20/32,5,1743,FECHADO,81,0,1422.4,1422.4,2020-07-27,1,ITAPETININGA - 1 - CLJ .,CLEBER JOSE MEDINA CUNHA (CLJ),CAPELA DO ALTO,SP,-23.4685,-47.7388,2020-06-22,06/2020,2020,2020-08-10,2020-08-01,2020,4.0,77.6,54.32
3,14306,20/32,20/39,5,1743,FECHADO,37,0,491.3,491.3,2020-09-14,1,ITAPETININGA - 1 - CLJ .,CLEBER JOSE MEDINA CUNHA (CLJ),CAPELA DO ALTO,SP,-23.4685,-47.7388,2020-08-10,08/2020,2020,2020-09-28,2020-09-01,2020,0.0,0.0,0.00
4,21573,20/39,20/46,5,1743,FECHADO,39,0,1291.0,1291.0,2020-11-09,1,ITAPETININGA - 1 - CLJ .,CLEBER JOSE MEDINA CUNHA (CLJ),CAPELA DO ALTO,SP,-23.4685,-47.7388,2020-09-28,09/2020,2020,2020-11-16,2020-11-01,2020,0.0,0.0,0.00
